**Open file and make data copy for further comparison**

In [1]:
import json

with open(f'E:/Peter/Scheme/coco-annotator/datasets/scheme_copy/.exports/coco-1621363362.794772.json', 'r+') as file:
    orig_data = json.load(file)

In [2]:
%%time
import copy

data = copy.deepcopy(orig_data)

Wall time: 131 ms


**Make functions and dicts**

In [3]:
image_id_shift = dict()
categories_id_shift = dict()
annotations_id_shift = dict()


def make_shift_dict(orig_list, new_id=0):
    id_shift_dict = dict()
    
    for e in orig_list:
        if e['id'] in id_shift_dict:
            raise KeyError('ID {} is in the dict!'.format(e['id']))
        else:
            id_shift_dict[e['id']] = new_id
            new_id += 1
            
    return id_shift_dict

def apply_shift(orig_list, shift_dict, key):
    for e in orig_list:
        if e[key] not in shift_dict:
            raise KeyError('ID {} not in the dict!'.format(e[key]))
        else:
            e[key] = shift_dict[e[key]]
    
    return orig_list

**Make shift dictionaries and apply them to our data.** 

Note that after shifting ids for images and categories in its dictionaries we must shift their ids everywhere in the exact same way.

In [4]:
image_id_shift = make_shift_dict(data['images'], 0)
categories_id_shift = make_shift_dict(data['categories'], 0)
annotations_id_shift = make_shift_dict(data['annotations'], 0)

data['images'] = apply_shift(data['images'], image_id_shift, 'id') # image ids
data['annotations'] = apply_shift(data['annotations'], annotations_id_shift, 'id') # annotation ids
data['categories'] = apply_shift(data['categories'], categories_id_shift, 'id') # category ids
data['annotaions'] = apply_shift(data['annotations'], image_id_shift, 'image_id') # image ids to annotations
data['annotations'] = apply_shift(data['annotations'], categories_id_shift, 'category_id') # category ids to annotations

**Write shifted dataset to a file**

In [5]:
with open(f'E:/Peter/Scheme/datasets/18-05-labels-s.json', 'w+') as file:
    json.dump(data, file)

**Simply check that nothing has gone wrong**

In [9]:
assert len(orig_data['images']) == len(data['images'])
assert len(orig_data['categories']) == len(data['categories'])
assert len(orig_data['annotations']) == len(data['annotations'])